# Homework 5

## FINM 37400 - 2023

### UChicago Financial Mathematics

* Mark Hendricks
* hendricks@uchicago.edu

# 1. Forward Rates

Use the data set `treasury_zero_coupon_2022-12-30.xlsx`.

It gives prices on **zero coupon bonds** with maturities of 1 through 5 years.
* These are prices per \\$1 face value on bonds that only pay principal.
* Such bonds can be created from treasuries by *stripping* out their coupons.
* In essence, you can consider these prices as the discount factors $Z$, for maturity intervals 1 through 5 years.

In this problem, we focus on six dates: the month of December in 2017 through 2022.

In [2]:
import pandas as pd
import numpy as np
import datetime
import warnings

import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (12,6)
plt.rcParams['font.size'] = 15
plt.rcParams['legend.fontsize'] = 13

from matplotlib.ticker import (MultipleLocator,
                               FormatStrFormatter,
                               AutoMinorLocator)
import sys
import treasury_cmds

In [3]:
treasury_zero_coupon = pd.read_excel('C:/Users/dcste/OneDrive/fixed_income/fixed_income_FORKED/finm-fixedincome-2023/data/treasury_zero_coupon_2022-12-30.xlsx')

In [4]:
treasury_zero_coupon

,date,1,2,3,4,5
0,1952-06-30,0.981778,0.962531,0.942467,0.926341,0.904183
1,1952-07-31,0.980948,0.960315,0.939312,0.923089,0.898516
2,1952-08-29,0.980578,0.959750,0.937819,0.921458,0.895717
3,1952-09-30,0.980312,0.959098,0.937551,0.920961,0.891676
4,1952-10-31,0.981108,0.959745,0.937583,0.919290,0.897248
...,...,...,...,...,...,...
842,2022-08-31,0.965955,0.933432,0.900600,0.871263,0.846584
843,2022-09-30,0.959839,0.918517,0.880655,0.846696,0.816731
844,2022-10-31,0.954288,0.914926,0.876564,0.841926,0.811304
845,2022-11-30,0.954375,0.918220,0.887608,0.857356,0.830593


## 1.1

For each of the six dates, (Dec 2017, Dec 2018, ...), calculate the forward discount factor, $F(t,T_1,T_2)$, where $T_2 = T_1 + 1$ for $1\le T_1\le 4$.

Make a plot of the curve $F$ at each of these six dates.

## 1.2



Use $F$ above to calculate the continuously compounded forward rates, $f$, at the six dates.

Make a plot of the curve, $f$, at each of these six dates.

## 1.3

Calculate the continuously compounded spot rates, $r$, at the six dates.

Plot both the forward curve and the spot curve on the date of Dec 2017.

Recall that $f(t,t,t+1) = r(t,t+1)$.

## 1.4

Plot the forward curve on Dec 2017 against the time-series evolution of $r(t,t+1)$ for dates $t$ between Dec 2017 and Dec 2021.

How does the evolution of one-year rates between 2017-2021 compare to the forward rates of 2017?

# 2. Carry Trade

## 2.1

Suppose it is December 2017, and you determine to implement a carry trade with the following specification...

* Long \\$100 million of the 5-year zero-coupon bond (maturing Dec 2022.)
* Short \\$100 million of the 1-year zero-coupon bond (maturing Dec 2018.)
* Assume there is a 2% haircut on each side of the trade, so it requires \\$4 million of investor capital to initiate it.

Trace the profit and loss each year.
* Note that each year you will roll over the short position until the trade concludes in Dec 2022.

Calculate the total profit and loss when the trade concludes in Dec 2022.
* Calculate the total return on the initial \\$4 million of investor capital.

## 2.2

How would this trade play out if the path of one-year spot rates equaled the forward rates observed in 2017?

## 2.3

Given Fact 3 of the *dynamic* (conditional) tests of EH in notebook W.5., do you expect that as of Dec 2022 the long-short trade above looks more or less favorable as of Dec 2022-2027 than it did as of Dec 2017-2022?